# 🧠 Model Training - Deep Learning for Sign Language

This notebook builds and trains a deep neural network for sign language gesture classification.

## Objectives
- Load preprocessed data
- Build deep learning model architecture
- Configure training parameters and callbacks
- Train the model
- Visualize training progress
- Save the trained model

---

## 1. Import Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, 
    TensorBoard, CSVLogger
)

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

print("✅ Libraries imported successfully")
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

## 2. Load Processed Data

In [ ]:
# Load preprocessed data
PROCESSED_DIR = 'data/processed'
MODELS_DIR = 'models/saved_models'
LOGS_DIR = 'outputs/logs'
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

X_train = np.load(os.path.join(PROCESSED_DIR, 'X_train.npy'))
X_val = np.load(os.path.join(PROCESSED_DIR, 'X_val.npy'))
X_test = np.load(os.path.join(PROCESSED_DIR, 'X_test.npy'))
y_train = np.load(os.path.join(PROCESSED_DIR, 'y_train.npy'))
y_val = np.load(os.path.join(PROCESSED_DIR, 'y_val.npy'))
y_test = np.load(os.path.join(PROCESSED_DIR, 'y_test.npy'))

print("\n" + "="*60)
print("DATA LOADED")
print("="*60)
print(f"Training set:   {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set:       {X_test.shape}")
print("="*60)

In [ ]:
# Load label encoder
with open('models/label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

num_classes = len(label_encoder.classes_)

print(f"\nNumber of classes: {num_classes}")
print(f"Classes: {label_encoder.classes_}")

In [ ]:
# Convert labels to categorical
y_train_cat = to_categorical(y_train, num_classes)
y_val_cat = to_categorical(y_val, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

print(f"\nCategorical labels shape:")
print(f"  Training:   {y_train_cat.shape}")
print(f"  Validation: {y_val_cat.shape}")
print(f"  Test:       {y_test_cat.shape}")

## 3. Build Model Architecture

In [ ]:
def create_model(input_shape, num_classes, model_type='dense'):
    """
    Create a deep neural network for sign language classification.
    
    Args:
        input_shape (int): Number of input features
        num_classes (int): Number of output classes
        model_type (str): 'dense' or 'deep_dense'
    
    Returns:
        keras.Model: Compiled model
    """
    if model_type == 'dense':
        # Standard dense network
        model = models.Sequential([
            layers.Input(shape=(input_shape,)),
            
            # First block
            layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            
            # Second block
            layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            
            # Third block
            layers.Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
            layers.BatchNormalization(),
            layers.Dropout(0.2),
            
            # Output layer
            layers.Dense(num_classes, activation='softmax')
        ], name='SignLanguageClassifier')
    
    elif model_type == 'deep_dense':
        # Deeper network for more complex patterns
        model = models.Sequential([
            layers.Input(shape=(input_shape,)),
            
            layers.Dense(256, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.4),
            
            layers.Dense(128, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            
            layers.Dense(64, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            
            layers.Dense(32, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.2),
            
            layers.Dense(num_classes, activation='softmax')
        ], name='DeepSignLanguageClassifier')
    
    return model

In [ ]:
# Create model
input_shape = X_train.shape[1]
MODEL_TYPE = 'dense'  # Change to 'deep_dense' for deeper network

model = create_model(input_shape, num_classes, model_type=MODEL_TYPE)

# Display model architecture
print("\n" + "="*60)
print("MODEL ARCHITECTURE")
print("="*60)
model.summary()
print("="*60)

## 4. Compile Model

In [ ]:
# Compile model
LEARNING_RATE = 0.001

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

print("\n✅ Model compiled successfully")
print(f"   Optimizer: Adam (lr={LEARNING_RATE})")
print(f"   Loss: Categorical Crossentropy")
print(f"   Metrics: Accuracy, Precision, Recall")

## 5. Configure Training Callbacks

In [ ]:
# Create timestamp for this training run
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = f"sign_language_model_{timestamp}"

# Define callbacks
callbacks = [
    # Save best model
    ModelCheckpoint(
        filepath=os.path.join(MODELS_DIR, 'best_model.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    
    # Early stopping
    EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Reduce learning rate on plateau
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=7,
        min_lr=1e-7,
        verbose=1
    ),
    
    # TensorBoard logging
    TensorBoard(
        log_dir=os.path.join(LOGS_DIR, model_name),
        histogram_freq=1,
        write_graph=True
    ),
    
    # CSV logging
    CSVLogger(
        filename=os.path.join(LOGS_DIR, f'{model_name}.csv'),
        separator=',',
        append=False
    )
]

print("\n✅ Callbacks configured:")
print("   - ModelCheckpoint (save best model)")
print("   - EarlyStopping (patience=20)")
print("   - ReduceLROnPlateau (factor=0.5, patience=7)")
print("   - TensorBoard logging")
print("   - CSV logging")

## 6. Train Model

In [ ]:
# Training parameters
EPOCHS = 150
BATCH_SIZE = 32

print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)
print(f"Epochs: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Training samples: {X_train.shape[0]}")
print(f"Validation samples: {X_val.shape[0]}")
print(f"Steps per epoch: {X_train.shape[0] // BATCH_SIZE}")
print("="*60 + "\n")

In [ ]:
# Train model
history = model.fit(
    X_train, y_train_cat,
    validation_data=(X_val, y_val_cat),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*60)
print("✅ TRAINING COMPLETED!")
print("="*60)

## 7. Visualize Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Accuracy
axes[0, 0].plot(history.history['accuracy'], label='Train', linewidth=2)
axes[0, 0].plot(history.history['val_accuracy'], label='Validation', linewidth=2)
axes[0, 0].set_xlabel('Epoch', fontsize=11)
axes[0, 0].set_ylabel('Accuracy', fontsize=11)
axes[0, 0].set_title('Model Accuracy', fontsize=12, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(alpha=0.3)

# Loss
axes[0, 1].plot(history.history['loss'], label='Train', linewidth=2)
axes[0, 1].plot(history.history['val_loss'], label='Validation', linewidth=2)
axes[0, 1].set_xlabel('Epoch', fontsize=11)
axes[0, 1].set_ylabel('Loss', fontsize=11)
axes[0, 1].set_title('Model Loss', fontsize=12, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(alpha=0.3)

# Precision
axes[1, 0].plot(history.history['precision'], label='Train', linewidth=2)
axes[1, 0].plot(history.history['val_precision'], label='Validation', linewidth=2)
axes[1, 0].set_xlabel('Epoch', fontsize=11)
axes[1, 0].set_ylabel('Precision', fontsize=11)
axes[1, 0].set_title('Model Precision', fontsize=12, fontweight='bold')
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(alpha=0.3)

# Recall
axes[1, 1].plot(history.history['recall'], label='Train', linewidth=2)
axes[1, 1].plot(history.history['val_recall'], label='Validation', linewidth=2)
axes[1, 1].set_xlabel('Epoch', fontsize=11)
axes[1, 1].set_ylabel('Recall', fontsize=11)
axes[1, 1].set_title('Model Recall', fontsize=12, fontweight='bold')
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/visualizations/training_history.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Training Summary

In [ ]:
# Get best metrics
best_epoch = np.argmax(history.history['val_accuracy'])
best_val_acc = max(history.history['val_accuracy'])
best_val_loss = history.history['val_loss'][best_epoch]
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]

print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)
print(f"Total epochs trained: {len(history.history['accuracy'])}")
print(f"Best epoch: {best_epoch + 1}")
print(f"\nBest Validation Metrics:")
print(f"  Accuracy:  {best_val_acc*100:.2f}%")
print(f"  Loss:      {best_val_loss:.4f}")
print(f"\nFinal Training Metrics:")
print(f"  Train Accuracy:      {final_train_acc*100:.2f}%")
print(f"  Validation Accuracy: {final_val_acc*100:.2f}%")
print("="*60)

## 9. Save Training Metadata

In [ ]:
# Create training metadata
training_metadata = {
    'training_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'model_name': model_name,
    'model_type': MODEL_TYPE,
    'total_epochs': len(history.history['accuracy']),
    'best_epoch': int(best_epoch + 1),
    'batch_size': BATCH_SIZE,
    'learning_rate': LEARNING_RATE,
    'best_val_accuracy': float(best_val_acc),
    'best_val_loss': float(best_val_loss),
    'final_train_accuracy': float(final_train_acc),
    'final_val_accuracy': float(final_val_acc),
    'num_classes': num_classes,
    'classes': label_encoder.classes_.tolist(),
    'train_samples': int(X_train.shape[0]),
    'val_samples': int(X_val.shape[0]),
    'test_samples': int(X_test.shape[0])
}

# Save metadata
os.makedirs('outputs/metrics', exist_ok=True)
with open(f'outputs/metrics/training_metadata_{timestamp}.json', 'w') as f:
    json.dump(training_metadata, f, indent=4)

print("\n✅ Training metadata saved!")
print(f"   File: outputs/metrics/training_metadata_{timestamp}.json")

## 10. Quick Test on Test Set

In [ ]:
# Load best model
best_model = keras.models.load_model(os.path.join(MODELS_DIR, 'best_model.keras'))

# Evaluate on test set
test_results = best_model.evaluate(X_test, y_test_cat, verbose=0)

print("\n" + "="*60)
print("QUICK TEST SET EVALUATION")
print("="*60)
print(f"Test Loss:      {test_results[0]:.4f}")
print(f"Test Accuracy:  {test_results[1]*100:.2f}%")
print(f"Test Precision: {test_results[2]*100:.2f}%")
print(f"Test Recall:    {test_results[3]*100:.2f}%")
print("="*60)
print("\n📝 For detailed evaluation, proceed to 05_model_evaluation.ipynb")

---

## 🎯 Summary

Model training completed successfully!

### What Was Done:
- ✅ Built deep neural network architecture
- ✅ Configured training with callbacks
- ✅ Trained model with early stopping
- ✅ Visualized training progress
- ✅ Saved best model and metadata
- ✅ Quick test set evaluation

### What's Next?
Proceed to **05_model_evaluation.ipynb** to:
- Perform comprehensive model evaluation
- Generate confusion matrix
- Analyze per-class performance
- Create detailed classification reports

### TensorBoard
To view training logs in TensorBoard:
```bash
tensorboard --logdir=outputs/logs
```

---